In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'twitter-svc-tuner-200326-1209'

In [2]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

40 training jobs have completed


In [3]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob', None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2020, 3, 26, 14, 11, 50, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'accuracy',
                                                 'Value': 0.7861875295639038},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2020, 3, 26, 14, 22, 19, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:159307201141:training-job/twitter-svc-tuner-200326-1209-038-423bf637',
 'TrainingJobName': 'twitter-svc-tuner-200326-1209-038-423bf637',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2020, 3, 26, 14, 15, 1, tzinfo=tzlocal()),
 'TunedHyperParameters': {'C': '0.45749319654611337',
                          'ngram_range': '2',
                          'smooth_idf': '"false"',
                          'sublinear_tf': '"false"'}}


In [4]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 40
{'lowest': 0.7728031277656555, 'highest': 0.7861875295639038}


,C,ngram_range,smooth_idf,sublinear_tf,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
20,100.000000,2.0,"""true""","""false""",twitter-svc-tuner-200326-1209-020-19750e6e,Completed,0.786188,2020-03-26 13:11:22-05:00,2020-03-26 13:18:08-05:00,406.0
8,0.076048,2.0,"""false""","""true""",twitter-svc-tuner-200326-1209-032-37af50bc,Completed,0.786188,2020-03-26 13:53:49-05:00,2020-03-26 14:00:53-05:00,424.0
22,0.000977,2.0,"""false""","""true""",twitter-svc-tuner-200326-1209-018-2bb80c3b,Completed,0.786188,2020-03-26 13:01:07-05:00,2020-03-26 13:07:59-05:00,412.0
16,96.951943,2.0,"""false""","""false""",twitter-svc-tuner-200326-1209-024-08341859,Completed,0.786188,2020-03-26 13:21:35-05:00,2020-03-26 13:28:55-05:00,440.0
23,0.000356,2.0,"""false""","""true""",twitter-svc-tuner-200326-1209-017-50cf4c1a,Completed,0.786188,2020-03-26 13:00:35-05:00,2020-03-26 13:07:57-05:00,442.0
1,0.084315,2.0,"""true""","""true""",twitter-svc-tuner-200326-1209-039-7a345b7f,Completed,0.786188,2020-03-26 14:14:49-05:00,2020-03-26 14:21:37-05:00,408.0
21,0.001702,2.0,"""false""","""true""",twitter-svc-tuner-200326-1209-019-b86b2ac2,Completed,0.786188,2020-03-26 13:00:55-05:00,2020-03-26 13:07:43-05:00,408.0
31,0.000160,2.0,"""true""","""true""",twitter-svc-tuner-200326-1209-009-0bcaf597,Completed,0.786188,2020-03-26 12:27:37-05:00,2020-03-26 12:34:24-05:00,407.0
3,100.000000,2.0,"""true""","""true""",twitter-svc-tuner-200326-1209-037-e8034dfb,Completed,0.786188,2020-03-26 14:03:48-05:00,2020-03-26 14:10:35-05:00,407.0
2,0.457493,2.0,"""false""","""false""",twitter-svc-tuner-200326-1209-038-423bf637,Completed,0.786188,2020-03-26 14:15:01-05:00,2020-03-26 14:22:19-05:00,438.0


In [7]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

In [8]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))